<a href="https://colab.research.google.com/github/nah90/CS777_PySparkLinearReg/blob/main/MET_CS_777_Assignment_3_Horak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --ignore-installed -q pyspark==3.2.1

In [ ]:
import os
import sys
import requests
from operator import add

from pyspark.sql import SparkSession

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *
import numpy as np

#Exception Handling and removing wrong data lines
def isfloat(value):
    try:
        float(value)
        return True 
    except:
         return False
         
# Added latitude/longitude filtering

# checking if the trip distance and fare amount is a float number
# checking if the trip duration is more than a minute, trip distance is more than 0.1 miles,
# checking latitude/longitude 
# fare amount and total amount are more than 0.1 dollars

# remove all taxi data with > 600$ and < 1$
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[4])> 60 and float(p[5])>0.10 and float(p[6])!=0 and float(p[7])!=0 and float(p[8])!=0 and float(p[9])!=0 and float(p[11])> 0.10 and float(p[16])> 0.10):
              if(float(p[11]) <= 600 and float(p[11]) >= 1):
                return p

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
# Set your file path here

! wget -q https://storage.googleapis.com/met-cs-777-data/taxi-data-sorted-small.csv.bz2


taxiFile = "taxi-data-sorted-small.csv.bz2"
#taxiFile = 'gs://met-cs-777-data/taxi-data-sorted-large.csv.bz2'
taxiDF = spark.read.format('csv').options(header='false', inferSchema='false',  sep =",").load(taxiFile)

taxilines_rdd = taxiDF.limit(200).rdd.map(tuple)
#taxilines_rdd = taxiDF.limit.rdd.map(tuple) #Works with full dataset too
taxilinesCorrected = taxilines_rdd.filter(correctRows)

In [ ]:
#Task 1
rdd1 = taxilinesCorrected.map(lambda x: (float(x[5]), float(x[11])))

In [ ]:
#linear reg calcs
lrc = rdd1.map(lambda x: np.array([x[0]*x[0], x[0]*x[1], x[0], x[1], 1])).reduce(add)

m_hat = (lrc[4]*lrc[1] - (lrc[2]*lrc[3]))/(lrc[4] * lrc[0] - (lrc[2]**2))
b_hat = ((lrc[0]*lrc[3]) - (lrc[2]*lrc[1]))/(lrc[4] * lrc[0] - (lrc[2]**2))

print(f"m = {m_hat}, b = {b_hat}")

m = 2.4573454657452714, b = 3.171696295716903


In [ ]:
#Task 2
taxiDFf = taxilinesCorrected.map(tuple).map(lambda x: (float(x[5]), float(x[11])))
taxi2DF = spark.createDataFrame(taxiDFf, schema= ['Distance', 'Fare Amount'])

rdd2 = taxi2DF.rdd.map(lambda x: (float(x[1]), np.array([1, float(x[0])])))
rdd2.cache()

PythonRDD[29] at RDD at PythonRDD.scala:53

In [ ]:
#Gradient Descent
learningRate = 0.000000075
num_iteration = 50
cost = np.array([np.Inf, np.Inf]) #Cost array of size 2 for Bold Driver, start with np.Inf
beta = np.array([0.1, 0.1])
size = float(rdd2.count())

for i in range(num_iteration):  

    gradientCost=rdd2.map(lambda x: (x[1], (x[0] - (np.dot(x[1], beta)))))\
                           .map(lambda x: np.array([x[0]*x[1], x[1]**2])).treeReduce(add)
   
    cost[1] = cost[0]
    cost[0]= gradientCost[1]

    if i > 0: #Don't need to adjust learning rate for first iteration as there is no previous cost to compare it to
      if cost[0] < cost[1]:
        learningRate = learningRate * 1.05
      elif cost[0] > cost[1]:
        learningRate = learningRate * 0.5
    
    gradient = (-1/size) * gradientCost[0]

    beta = beta - learningRate * gradient

    print(i+1, 'm = ', beta[1], 'b = ', beta[0]," Cost = ", cost[0])

rdd2.unpersist()

1 m =  0.10000074736110527 b =  0.10000046332017544  Cost =  7330.920667999998
2 m =  0.10000153209006998 b =  0.10000094980624562  Cost =  7330.917142143648
3 m =  0.10000235605526704 b =  0.10000146061649359  Cost =  7330.913439996322
4 m =  0.10000322121848593 b =  0.10000199696711536  Cost =  7330.909552743656
5 m =  0.10000412963960335 b =  0.1000025601351154  Cost =  7330.905471130594
6 m =  0.10000508348148733 b =  0.10000315146134697  Cost =  7330.901185439337
7 m =  0.10000608501514655 b =  0.10000377235370438  Cost =  7330.896685466241
8 m =  0.10000713662513706 b =  0.10000442429047487  Cost =  7330.891960497482
9 m =  0.1000082408152394 b =  0.10000510882385813  Cost =  7330.88699928359
10 m =  0.10000940021441941 b =  0.10000582758366162  Cost =  7330.881790012643
11 m =  0.10001061758308717 b =  0.10000658228118087  Cost =  7330.876320282168
12 m =  0.10001189581966875 b =  0.10000737471327352  Cost =  7330.8705770695915
13 m =  0.10001323796750661 b =  0.1000082067666372

PythonRDD[29] at RDD at PythonRDD.scala:53

In [ ]:
#Task 3

taxiDFf = taxilinesCorrected.map(tuple).map(lambda x: (float(x[4]), float(x[5]), float(x[11]), float(x[12]), float(x[16])))
taxi3DF = spark.createDataFrame(taxiDFf, schema= ['Duration', 'Distance', 'Fare Amount', 'Bridge/Tunnel Toll', 'Total Paid'])

rdd3  = taxi3DF.rdd.map(lambda x: (float(x[4]), np.array([1, float(x[0]), float(x[1]), float(x[2]), float(x[3])])))
rdd3.cache()

PythonRDD[90] at RDD at PythonRDD.scala:53

In [ ]:
#Gradient Descent
learningRate = 0.00000005
num_iteration = 50
cost = np.array([np.Inf, np.Inf]) #Cost array of size 2 for Bold Driver, start with np.Inf
beta = np.array([0, 0, 0, 0, 0]) #Four independent variables
size = float(rdd3.count())

for i in range(num_iteration):

    gradientCost=rdd3.map(lambda x: (x[1], (x[0] - (np.dot(x[1], beta)))))\
                           .map(lambda x: np.array([x[0]*x[1], x[1]**2])).treeReduce(add)
                           
    cost[1] = cost[0]
    cost[0]= gradientCost[1]

    if i > 0: #Don't need to adjust learning rate for first iteration as there is no previous cost to compare it to
      if cost[0] < cost[1]:
        learningRate = learningRate * 1.05
      elif cost[0] > cost[1]:
        learningRate = learningRate * 0.5
    
    gradient = (-1/size) * gradientCost[0]

    beta = beta - learningRate * gradient

    print(i+1, 'm = ', beta[1:], 'b = ', beta[0]," Cost = ", cost[0])


1 m =  [1.35043684e-04 6.19841228e-07 2.79933480e-06 1.97304094e-07] b =  3.946081871345029e-07  Cost =  11740.179599999998
2 m =  [2.76045561e-04 1.26724927e-06 5.72290942e-06 4.03379130e-07] b =  8.067582606766487e-07  Cost =  11615.731747867829
3 m =  [4.23227057e-04 1.94327252e-06 8.77542094e-06 6.18558251e-07] b =  1.2371165018954758e-06  Cost =  11486.53890084593
4 m =  [5.76813572e-04 2.64898118e-06 1.19616607e-05 8.43181470e-07] b =  1.6863629398103765e-06  Cost =  11352.496594727325
5 m =  [7.37034060e-04 3.38546566e-06 1.52865067e-05 1.07759517e-06] b =  2.1551903366285567e-06  Cost =  11213.506063549947
6 m =  [9.04120542e-04 4.15383475e-06 1.87549149e-05 1.32215150e-06] b =  2.6443029920805555e-06  Cost =  11069.47527046005
7 m =  [1.07830754e-03 4.95521344e-06 2.23719089e-05 1.57720768e-06] b =  3.1544153504115397e-06  Cost =  10920.320017627471
8 m =  [1.25983145e-03 5.79074040e-06 2.61425681e-05 1.84312520e-06] b =  3.6862503930559454e-06  Cost =  10765.965136166986
9 m 

In [ ]:
spark.stop(); sc.stop()